In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
from scipy.stats import kurtosis
from scipy.stats import skew
from sklearn.model_selection import train_test_split
import statsmodel.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import eli5
from eli5.sklearn import PermutationImportance
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score,roc_curve,auc
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

In [ ]:
os.chdir("D:/edwisor/project 2")

In [ ]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
#DROPPING UNNECESSARY VARIABLES
train=train.drop(['ID_code'],axis=1)

In [ ]:
#CONVERTING REQUIRED DATA TYPES
train.dtypes
train['target']=train['target'].astype('object')

In [ ]:
#CHECKING MISSING VALUE
train.isnull().sum()
test.isnull().sum()         #no missing value found

In [ ]:
#CHECKING TARGET CLASS IMBALANCE
train['target'].value_counts()

In [ ]:
#DISTRIBUTION OF VARIABLES IN TRAIN DATA W.R.T TARGET CLASS

def plot_train_attribute_distribution(t0,t1,label1,label2,train_attributes):
    i=0
    sns.set_style('whitegrid')
    fig=plt.figure()
    ax=plt.subplots(10,10,figsize=(22,18))
    for attribute in train_attributes:
        i+=1
        plt.subplot(10,10,i)
        sns.distplot(t0[attribute],hist=False,label=label1)
        sns.distplot(t1[attribute],hist=False,label=label2)
        plt.legend()
        plt.xlabel('Attribute')
        sns.set_style("ticks", {"xtick.major.size":8,"ytick.major.size":8})
        plt.show()

In [ ]:
t0=train[train.target.values==0]
t1=train[train.target.values==1]

In [ ]:
#distribution of top 100 numerics variables in train data
train_attributes=train.columns.values[1:101]
plot_train_attribute_distribution(t0,t1,'0','1',train_attributes)

In [ ]:
#distribution of next 100 numeric variables in train data
train_attributes=train.columns.values[101:202]
plot_train_attribute_distribution(t0,t1,'0','1',train_attributes)

In [ ]:
#DISTRIBUTION OF VARIABLES IN  TEST DATA
def plot_test_attribute_distribution(test_attributes):
    i=0
    sns.set_style('whitegrid')
    fig=plt.figure()
    ax=plt.subplots(10,10,figsize=(22,18))
    for attribute in test_attributes:
        i+=1
        plt.subplot(10,10,i)
        sns.distplot(test[attribute],hist=False)
        plt.xlabel('Attribute')
        sns.set_style("ticks", {"xtick.major.size":8,"ytick.major.size":8})
        plt.show()

In [ ]:
#distribution of top 100 numeric variables in test data
test_attributes=test.columns.values[1:101]
plot_test_attribute_distribution(test_attributes)

In [ ]:
#distribution of another 100 numeric variables in test data
test_attributes=test.columns.values[101:202]
plot_test_attribute_distribution(test_attributes)

In [ ]:
#OUTLIER ANALYSIS

#in train data
cnames=train.iloc[:,1:202].columns
for i in cnames:
    q75, q25=np.percentile(train[i],[75,25])
    iqr=q75-q25
    min=q25-(iqr*1.5)
    max=q75+(iqr*1.5)
    train=train.drop(train[train.loc[:,i]<min].index)
    train=train.drop(train[train.loc[:,i]>max].index)

In [ ]:
#in test data
cnames=test.iloc[:,1:202].columns
for i in cnames:
    q75, q25=np.percentile(test[i],[75,25])
    iqr=q75-q25
    min=q25-(iqr*1.5)
    max=q75+(iqr*1.5)
    test=test.drop(test[test.loc[:,i]<min].index)
    test=test.drop(test[test.loc[:,i]>max].index)

In [ ]:
#CHECKING MULTICOLLINEARITY IN TRAIN DATA

train_attributes=train.columns.values[1:202]
train_correlations=train[train_attributes].corr().abs().unstack().sort_values(kind='quicksort').reset_index()
train_correlations=train_correlations[train_correlations['level_0']!=train_correlations['level_1']]
print(train_correlations.head(10))
print(train_correlations.tail(10))

In [ ]:
#PRINCIPAL COMPONENT ANALYSIS

data=train._get_numeric_data()
x=data.values
x=scale(x)
matrix=PCA(n_components=200)
matrix.fit(x)
variance=matrix.explained_variance_ratio_
var=np.cumsum(np.round(matrix.explained_variance_ratio_,decimals=3)*100)
plt.ylabel('% variance explained')
plt.xlabel('Features')
plt.plot(var)

In [ ]:
#FEATURE ENGINEERING

train['mean']=train.iloc[:,1:201].apply(np.mean,axis=1)
train['median']=train.iloc[:,1:201].apply(np.median,axis=1)
train['minimum']=train.iloc[:,1:201].apply(np.min,axis=1)
train['maximum']=train.iloc[:,1:201].apply(np.max,axis=1)
train['stdeviation']=train.iloc[:,1:201].apply(np.std,axis=1)
train['skewness']=train.iloc[:,1:201].apply(skew,axis=1)
train['kurtosis']=train.iloc[:,1:201].apply(kurtosis,axis=1)

In [ ]:
#plotting distribution
train_attributes=train.columns.values[201:208]
plot_train_attribute_distribution(t0,t1,'0','1',train_attributes)

In [ ]:
test['mean']=test.iloc[:,1:201].apply(np.mean,axis=1)
test['median']=test.iloc[:,1:201].apply(np.median,axis=1)
test['minimum']=test.iloc[:,1:201].apply(np.min,axis=1)
test['maximum']=test.iloc[:,0:201].apply(np.max,axis=1)
test['stdeviation']=test.iloc[:,0:201].apply(np.std,axis=1)
test['skewness']=test.iloc[:,0:201].apply(skew,axis=1)
test['kurtosis']=test.iloc[:,0:201].apply(kurtosis,axis=1)

In [ ]:
#plotting distribution
test_attributes=test.columns.values[201:208]
plot_test_attribute_distribution(test_attributes)

In [ ]:
# SPLITTING DATA
train_df,test_df=train_test_split(train,test_size=0.2)

In [ ]:
#Target balancing using SMOTE
sm=SMOTE(random_state=42,ratio=1.0)
x_smote,y_smote=sm.fit_sample(train_df[:,1:208],train_df[:,0])

In [ ]:
#Target balancing using ROSE
ros=RandomOverSampler()
x_rose,y_rose=ros.fit_sample(train_df[:,1:208],train_df[:,0])

In [ ]:
# LOGISTIC REGRESSION

#1. using target class imbalance
model_lr=sm.Logit(train_df['target'],train_df.iloc[:,1:208]).fit()
test_df['actual_prob']=model_lr.predict(test_df[:,1:208])
test_df['actual_val']=1
test_df.loc[test_df.actual_prob<0.5,'actual_val']=0
CM_lr=pd.crosstab(test_df['target'],test_df['actual_val'])
CM_lr

In [ ]:
#calculating auc

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],test_df['actual_val'])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
test_df=test_df.drop(['actual_prob','actual_val'],axis=1)

In [ ]:
#2. Logistic Regression using SMOTE
model_lr_s=sm.Logit(y_smote,x_smote).fit()
test_df['actual_prob']=model_lr_s.predict(test_df[:,1:208])
test_df['actual_val']=1
test_df.loc[test_df.actual_prob<0.5,'actual_val']=0
CM_lr_s=pd.crosstab(test_df['target'],test_df['actual_val'])
CM_lr_s

In [ ]:
#calculating auc

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],test_df['actual_val'])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
test_df=test_df.drop(['actual_prob','actual_val'],axis=1)

In [ ]:
#3. Logistic Regression using ROSE
model_lr_r=sm.Logit(y_rose,x_rose).fit()
test_df['actual_prob']=model_lr_r.predict(test_df[:,1:208])
test_df['actual_val']=1
test_df.loc[test_df.actual_prob<0.5,'actual_val']=0
CM_lr_r=pd.crosstab(test_df['target'],test_df['actual_val'])
CM_lr_r

In [ ]:
#calculating auc

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],test_df['actual_val'])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
test_df=test_df.drop(['actual_prob','actual_val'],axis=1)

In [ ]:
#RANDOM FOREST

train_df['target']=train_df['target'].replace(1,'Yes')
train_df['target']=train_df['target'].replace(0,'No')
test_df['target']=test_df['target'].replace(1,'Yes')
test_df['target']=test_df['target'].replace(0,'No')

#1. using target class imbalance
model_rf=RandomForestClassifier(n_estimators=100).fit(train_df.iloc[:,1:208],train_df.iloc[:,0])
pred_rf=model_rf.predict(test_df.iloc[:,1:208])
CM_rf=pd.crosstab(test_df.iloc[:,0],pred_rf)
CM_rf

In [ ]:
#extracting important variables from random forest
perm_imp=PermutationImportance(model_rf,random_state=42)
perm_imp.fit(train_df.iloc[:,1:208],train_df.iloc[:,0])
eli5.show_weights(perm_imp,feature_names=train_df.iloc[:,1:208].columns.tolist(),top=200)

In [ ]:
#calculating auc
test_df['target']=test_df['target'].replace('Yes',1)
test_df['target']=test_df['target'].replace('No',0)
pred_rf=pd.DataFrame(pred_rf)
pred_rf=pred_rf.replace('Yes',1)
pred_rf=pred_rf.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],pred_rf.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#2. Random forest using SMOTE

y_smote=y_smote.replace(1,'Yes')
y_smote=y_smote.replace(0,'No')
test_df['target']=test_df['target'].replace(1,'Yes')
test_df['target']=test_df['target'].replace(0,'No')

model_rf_s=RandomForestClassifier(n_estimators=100).fit(x_smote,y_smote)
pred_rf_s=model_rf_s.predict(test_df.iloc[:,1:208])
CM_rf_s=pd.crosstab(test_df.iloc[:,0],pred_rf_s)
CM_rf_s


In [ ]:
#calculating auc
test_df['target']=test_df['target'].replace('Yes',1)
test_df['target']=test_df['target'].replace('No',0)
pred_rf_s=pd.DataFrame(pred_rf_s)
pred_rf_s=pred_rf_s.replace('Yes',1)
pred_rf_s=pred_rf_s.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],pred_rf_s.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#2. Random forest using ROSE

y_rose=y_rose.replace(1,'Yes')
y_rose=y_rose.replace(0,'No')
test_df['target']=test_df['target'].replace(1,'Yes')
test_df['target']=test_df['target'].replace(0,'No')

model_rf_r=RandomForestClassifier(n_estimators=100).fit(x_rose,y_rose)
pred_rf_r=model_rf_r.predict(test_df.iloc[:,1:208])
CM_rf_r=pd.crosstab(test_df.iloc[:,0],pred_rf_r)
CM_rf_r

In [ ]:
#calculating auc
test_df['target']=test_df['target'].replace('Yes',1)
test_df['target']=test_df['target'].replace('No',0)
pred_rf_r=pd.DataFrame(pred_rf_r)
pred_rf_r=pred_rf_r.replace('Yes',1)
pred_rf_r=pred_rf_r.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],pred_rf_r.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#NAIVE BAYES

#1. using target class imbalance
test_df['target']=test_df['target'].replace(1,'Yes')
test_df['target']=test_df['target'].replace(0,'No')

model_nb=GaussianNB().fit(train_df.iloc[:,1:208],train_df.iloc[:,0])
pred_nb=model_nb.predict(test_df.iloc[:,1:208])
CM_nb=pd.crosstab(test_df.iloc[:,0],pred_nb)
CM_nb

In [ ]:
#calculating AUC
test_df['target']=test_df['target'].replace('Yes',1)
test_df['target']=test_df['target'].replace('No',0)
pred_nb=pd.DataFrame(pred_nb)
pred_nb=pred_nb.replace('Yes',1)
pred_nb=pred_nb.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],pred_nb.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#2. Naive Bayes using SMOTE

test_df['target']=test_df['target'].replace(1,'Yes')
test_df['target']=test_df['target'].replace(0,'No')

model_nb_s=GaussianNB().fit(x_smote,y_smote)
pred_nb_s=model_nb_s.predict(test_df.iloc[:,1:208])
CM_nb_s=pd.crosstab(test_df.iloc[:,0],pred_nb_s)
CM_nb_s

In [ ]:
#calculating AUC
test_df['target']=test_df['target'].replace('Yes',1)
test_df['target']=test_df['target'].replace('No',0)
pred_nb_s=pd.DataFrame(pred_nb_s)
pred_nb_s=pred_nb_s.replace('Yes',1)
pred_nb_s=pred_nb_s.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],pred_nb_s.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#2. Naive Bayes using ROSE

test_df['target']=test_df['target'].replace(1,'Yes')
test_df['target']=test_df['target'].replace(0,'No')

model_nb_r=GaussianNB().fit(x_rose,y_rose)
pred_nb_r=model_nb_r.predict(test_df.iloc[:,1:208])
CM_nb_r=pd.crosstab(test_df.iloc[:,0],pred_nb_r)
CM_nb_r

In [ ]:
#calculating AUC
test_df['target']=test_df['target'].replace('Yes',1)
test_df['target']=test_df['target'].replace('No',0)
pred_nb_r=pd.DataFrame(pred_nb_r)
pred_nb_r=pred_nb_r.replace('Yes',1)
pred_nb_r=pred_nb_r.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],pred_nb_r.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#XGBoost

#1. using target class imbalance
train_df['target']=train_df['target'].replace('Yes',1)
train_df['target']=train_df['target'].replace('No',0)
model_xgb=XGBClassifier()
model_xgb.fit(train_df.iloc[:,1:208],train_df.iloc[:,0])
pred_xgb=model_xgb.predict(test_df.iloc[:1:208])
preds_xgb=[round(value) for value in pred_xgb]
CM_xgb=pd.crosstab(test_df.iloc[:,0],preds_xgb)
CM_xgb

In [ ]:
#calculating auc

preds_xgb=pd.DataFrame(preds_xgb)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],preds_xgb.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#XGBoost

#2. XGBoost using SMOTE
y_smote=y_smote.replace('Yes',1)
y_smote=y_smote.replace('No',0)
model_xgb_s=XGBClassifier()
model_xgb_s.fit(x_smote,y_smote)
pred_xgb_s=model_xgb_s.predict(test_df.iloc[:1:208])
preds_xgb_s=[round(value) for value in pred_xgb_s]
CM_xgb_s=pd.crosstab(test_df.iloc[:,0],preds_xgb_s)
CM_xgb_s

In [ ]:
#calculating auc
preds_xgb_s=pd.DataFrame(preds_xgb_s)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],preds_xgb_s.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#3. XGBoost using ROSE
y_rose=y_rose.replace('Yes',1)
y_rose=y_rose.replace('No',0)
model_xgb_r=XGBClassifier()
model_xgb_r.fit(x_rose,y_rose)
pred_xgb_r=model_xgb_r.predict(test_df.iloc[:1:208])
preds_xgb_r=[round(value) for value in pred_xgb_r]
CM_xgb_r=pd.crosstab(test_df.iloc[:,0],preds_xgb_r)
CM_xgb_r

In [ ]:
#calculating auc
preds_xgb_r=pd.DataFrame(preds_xgb_r)

false_positive_rate,recall,thresholds=roc_curve(test_df.iloc[:,0],preds_xgb_r.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#KNN

cnames=train.iloc[:,1:208].columns
for i in cnames:
    train[i]=(train[i]-np.min(train[i]))/(np.max(train[i])-np.min(train[i]))

In [ ]:
train_data,test_data=train_test_split(train,test_size=0.2)

In [ ]:
#Target balancing using SMOTE
sm=SMOTE(random_state=42,ratio=1.0)
x_s,y_s=sm.fit_sample(train_data[:,1:208],train_data[:,0])

In [ ]:
#Target balancing using ROSE
ros=RandomOverSampler()
x_r,y_r=ros.fit_sample(train_data[:,1:208],train_data[:,0])

In [ ]:
train_data['target']=train_data['target'].replace(1,'Yes')
train_data['target']=train_data['target'].replace(0,'No')
test_data['target']=test_data['target'].replace(1,'Yes')
test_data['target']=test_data['target'].replace(0,'No')

In [ ]:
#1. using target class imbalance
model_knn=KNeighborsClassifier(n_neighbors=1).fit(train_data.iloc[:,1:208],train_data.iloc[:,0])
pred_knn=model_knn.predict(test_data.iloc[:,1:208])
CM_knn=pd.crosstab(test_data.iloc[:,0],pred_knn)
CM_knn

In [ ]:
#calculating auc

test_data['target']=test_data['target'].replace('Yes',1)
test_data['target']=test_data['target'].replace('No',0)
pred_knn=pd.DataFrame(pred_knn)
pred_knn=pred_knn.replace('Yes',1)
pred_knn=pred_knn.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_data.iloc[:,0],pred_knn.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
# 2. KNN using SMOTE

y_s=y_s.replace(1,'Yes')
y_s=y_s.replace(0,'No')
test_data['target']=test_data['target'].replace(1,'Yes')
test_data['target']=test_data['target'].replace(0,'No')

In [ ]:
model_knn_s=KNeighborsClassifier(n_neighbors=1).fit(x_s,y_s)
pred_knn_s=model_knn_s.predict(test_data.iloc[:,1:208])
CM_knn_s=pd.crosstab(test_data.iloc[:,0],pred_knn_s)
CM_knn_s

In [ ]:
#calculating auc

test_data['target']=test_data['target'].replace('Yes',1)
test_data['target']=test_data['target'].replace('No',0)
pred_knn_s=pd.DataFrame(pred_knn_s)
pred_knn_s=pred_knn_s.replace('Yes',1)
pred_knn_s=pred_knn_s.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_data.iloc[:,0],pred_knn_s.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
# 3. KNN using ROSE

y_r=y_r.replace(1,'Yes')
y_r=y_r.replace(0,'No')
test_data['target']=test_data['target'].replace(1,'Yes')
test_data['target']=test_data['target'].replace(0,'No')

model_knn_r=KNeighborsClassifier(n_neighbors=1).fit(x_r,y_r)
pred_knn_r=model_knn_r.predict(test_data.iloc[:,1:208])
CM_knn_r=pd.crosstab(test_data.iloc[:,0],pred_knn_r)
CM_knn_r

In [ ]:
#calculating auc

test_data['target']=test_data['target'].replace('Yes',1)
test_data['target']=test_data['target'].replace('No',0)
pred_knn_r=pd.DataFrame(pred_knn_r)
pred_knn_r=pred_knn_r.replace('Yes',1)
pred_knn_r=pred_knn_r.replace('No',0)

false_positive_rate,recall,thresholds=roc_curve(test_data.iloc[:,0],pred_knn_r.iloc[:,0])
roc_auc=auc(false_positive_rate,recall)
print('AUC:',roc_auc)

In [ ]:
#FINAL PREDICTION USING LOGISTIC REGRESSION ROSE

test['actual_prob']=model_lr_r.predict(test[:,1:208])
test['actual_val']=1
test.loc[test.actual_prob<0.5,'actual_val']=0